In [1]:
from bs4 import BeautifulSoup as BS
import urllib.request as req
import urllib.parse as pa
import pandas as pd
from tqdm import tqdm_notebook
import time
import re
import math

In [2]:
# 평점 url 얻기 위한 url_open

def url_point_Open():
    url = 'https://movie.naver.com/movie/bi/mi/point.nhn?code=129049'
    url_open = req.urlopen(url)
    
    return url_open

In [3]:
def url_Open(sub_url, page_num):
    url = 'https://movie.naver.com' + sub_url +'&page='+ str(page_num)
    url_open = req.urlopen(url)
    
    return url_open

In [4]:
soup = BS(url_point_Open(), 'html.parser')
ifr_url = soup.find('iframe')['src']
ifr_url

'/movie/bi/mi/pointWriteFormList.nhn?code=129049&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'

In [5]:
soup = BS(url_Open(ifr_url, 11), 'html.parser')

In [6]:
count = soup.find('strong', 'total').find('em')
count

<em>14,624</em>

In [7]:
count = int(count.string.replace(',', ''))

In [8]:
count

14624

In [9]:
loop_count = math.ceil(count/10)
loop_count

1463

In [10]:
point = []
review = []
date = []

In [11]:
page_num = 1

while page_num  < loop_count+1:
    soup = BS(url_Open(ifr_url, page_num), 'html.parser')
    result = soup.find(class_='score_result')
    lis = result.find_all('li')
    print(page_num)
    
    for lst in lis:
        count_span = len(lst.find(class_='score_reple').find_all('span'))
        count_icon = len(lst.find_all(class_='ico_viewer'))
        count_spo = len(lst.find_all(class_='text_spo'))
        point.append(int(lst.find(class_='star_score').find('em').get_text()))
        print(count_span, count_icon, count_spo)
        
        if count_icon ==1 and count_span == 2:
            r = lst.find(class_='score_reple').find('p').get_text().strip().replace('\n', '').replace('\r','').replace('\t', '').replace('관람객', '')
        elif count_icon ==1 and count_spo == 1:
            r = lst.find_all('span')[4].get_text().strip()
        elif count_icon == 1:
            r = lst.find(class_='score_reple').find('p').get_text().strip().replace('\n', '').replace('\r', '').replace('\t', '').replace('관람객', '')
        elif count_spo == 1:
            r = lst.find_all('span')[3].get_text().strip()
        elif count_span == 3:
            r = lst.find_all('a')[0]['data-src']
        elif count_span == 2:
            r = lst.find(class_='score_reple').find('p').get_text().strip()
#         else:
#             r = lst.find_all('a')[0]['data-src']
        review.append(r)
        date.append(lst.find(class_='score_reple').find_all('em')[1].get_text())
            
    page_num += 1


1
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
3 0 0
2 0 0
2 0 0
2 0 0
3 1 0
2
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
4 1 0
3 1 0
2 0 0
2 0 0
3 1 0
3
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
2 0 0
2 0 0
3 0 0
3 1 0
2 0 0
4
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
4 1 0
5
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
6
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
3 0 0
3 1 0
2 0 0
7
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 0 0
3 0 0
2 0 0
2 0 0
2 0 0
8
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
9
2 0 0
2 0 0
3 0 0
2 0 0
3 0 0
2 0 0
2 0 0
3 0 0
2 0 0
2 0 0
10
2 0 0
3 0 0
2 0 0
3 0 0
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
11
2 0 0
2 0 0
2 0 0
3 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
12
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 0 0
2 0 0
13
2 0 0
2 0 0
3 1 0
3 0 0
3 1 0
3 0 0
3 1 0
2 0 0
2 0 0
2 0 0
14
2 0 0
2 0 0
3 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
15
2 0 0
2 0 0
3 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
16
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
1

132
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
2 0 0
2 0 0
3 1 0
133
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
2 0 0
134
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
135
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
136
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
137
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 0 0
138
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
139
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
140
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
2 0 0
141
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
2 0 0
142
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
3 0 0
2 0 0
3 1 0
2 0 0
143
3 1 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
144
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
3 0 0
2 0 0
3 1 0
145
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
3 0 0
2 0 0
2 0 0
2 0 0
146
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
147
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0


261
3 0 0
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
2 0 0
2 0 0
262
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
263
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
2 0 0
2 0 0
4 1 0
2 0 0
2 0 0
264
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
265
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
266
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
2 0 0
267
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
268
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
269
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
270
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
271
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
272
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
273
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
274
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
275
3 1 0
3 1 0
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
276
3 0 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0


391
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
392
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
393
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
394
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
395
2 0 0
2 0 0
3 1 0
4 1 0
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
396
3 1 0
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
397
4 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
398
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 0 0
3 1 0
3 1 0
399
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
400
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
401
3 1 0
3 1 0
3 1 0
3 1 0
3 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
402
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
403
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
2 0 0
3 1 0
404
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
405
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
406
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0


521
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 0 0
522
2 0 0
3 1 0
3 1 0
2 0 0
2 0 0
3 0 0
2 0 0
2 0 0
2 0 0
2 0 0
523
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
524
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 0 0
2 0 0
2 0 0
525
3 0 0
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
526
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
2 0 0
2 0 0
3 1 0
527
3 1 0
3 0 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
528
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
529
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
530
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
531
3 1 0
3 1 0
3 1 0
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
532
3 1 0
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
533
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
2 0 0
534
3 0 0
3 0 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
535
3 1 0
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
536
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0


...

911
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
912
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
913
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
2 0 0
914
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
915
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
2 0 0
916
3 0 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
917
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
918
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
919
3 0 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
920
3 0 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
921
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
2 0 0
922
3 1 0
2 0 0
3 1 0
3 1 0
3 0 0
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
923
2 0 0
3 1 0
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
924
2 0 0
3 1 0
3 1 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
925
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
926
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0


1039
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
3 0 0
2 0 0
2 0 0
3 1 0
1040
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
1041
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
1042
3 1 0
3 1 0
2 0 0
2 0 0
3 0 0
3 1 0
3 1 0
2 0 0
2 0 0
2 0 0
1043
3 1 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
1044
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
1045
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
1046
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
1047
3 1 0
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
1048
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
1049
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
1050
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
1051
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
3 0 0
2 0 0
2 0 0
2 0 0
4 1 0
1052
3 1 0
3 1 0
3 1 0
3 1 0
3 0 0
2 0 0
4 1 0
2 0 0
3 1 0
2 0 0
1053
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
1054
2 0 0
2 0 0
2 0 0
2 

1166
3 0 0
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
1167
2 0 0
3 1 0
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
3 1 0
1168
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
1169
2 0 0
3 1 0
3 1 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
1170
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
2 0 0
1171
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
1172
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
4 1 0
2 0 0
2 0 0
3 1 0
1173
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
1174
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
1175
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
1176
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
1177
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
1178
2 0 0
3 1 0
4 1 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
1179
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
3 0 0
3 1 0
2 0 0
2 0 0
2 0 0
1180
3 1 0
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
1181
2 0 0
3 1 0
2 0 0
3 

3 1 0
1294
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
2 0 0
1295
3 1 0
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
1296
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
1297
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
1298
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
1299
3 1 0
3 0 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
2 0 0
1300
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
3 1 0
1301
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
3 1 0
1302
3 1 0
3 1 0
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
1303
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
3 1 0
2 0 0
2 0 0
2 0 0
3 1 0
1304
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
3 1 0
1305
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
3 1 0
1306
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
1307
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
1308
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
1309
3 0 0
3 1 0
2 

1421
3 1 0
2 0 0
2 0 0
3 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
1422
3 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
1423
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
3 0 0
2 0 0
2 0 0
1424
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
2 0 0
3 1 0
2 0 0
1425
3 1 0
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
1426
2 0 0
2 0 0
2 0 0
3 0 0
3 0 0
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
1427
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
3 1 0
3 0 0
1428
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
3 1 0
2 0 0
2 0 0
1429
2 0 0
3 1 0
3 1 0
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
3 1 0
2 0 0
1430
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
2 0 0
1431
3 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
2 0 0
3 1 0
3 1 0
1432
2 0 0
2 0 0
2 0 0
2 0 0
3 1 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
1433
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
1434
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
2 0 0
1435
2 0 0
2 0 0
2 0 0
2 0 0
3 0 0
2 0 0
3 1 0
2 0 0
2 0 0
2 0 0
1436
2 0 0
2 0 0
2 0 0
2 

In [12]:
df = pd.DataFrame(dict(zip(['날짜','리뷰', '평점'], [date, review, point])))
df

,날짜,리뷰,평점
0,2015.10.08 11:20,보면 감자먹고 싶어지는 영화,10
1,2015.10.08 11:32,화성판 삼시세끼...,8
2,2015.10.08 22:23,사람은일단똑똑해야된다,9
3,2015.10.08 01:12,최고의 영화. 다만 이 영화를 인터스텔라같은 영화라고 기대하고 보지 마시고 캐스트 ...,10
4,2015.10.08 08:51,내가더낫죠? 닐 암스트롱? 이말한마디로 지렷다..,10
...,...,...,...
14619,2018.08.10 12:24,인터스텔라 짜투리로 만든느낌. 멧데이는 조연급으로 나와야 영화가산다. 라이언일병부터...,2
14620,2015.10.08 10:43,"결말이 뻔하게 보여서 긴장감이 좀 떨어지고, 감동면에서나 위트면에서나 다소 엉성....",6
14621,2019.06.29 16:40,말도 안되고 개지루한 영화. 리들리스콧감독이라 기대했지만 인간은 늙으면 정신적으로도...,1
14622,2019.07.07 14:49,별 한개도 아까워 제발 보지마,1


In [13]:
df.to_csv('movie_Martian.csv', encoding='utf-8')